In [2]:
import glob
import pandas as pd
import re
import pandas as pd
from utils.calculate_results_helper import extract_params_from_file_name, refine_output_df

csv_files = glob.glob('data/evaluation_results/*.csv')
dfs = []
empty_n = 0
for file in csv_files:
    if file =='data/evaluation_results/joint_results.csv':
        continue
    tmp_df = pd.read_csv(file)
    if tmp_df.shape[0] == 0:
        print('EMPTY FILE: ', file)
        empty_n += 1
        continue
    print('SOURCE: ', file)
    tmp_df = extract_params_from_file_name(tmp_df)
    dfs.append(tmp_df)

res = pd.concat(dfs)
res = refine_output_df(res)
display(res)
res.to_csv('data/evaluation_results/joint_results.csv')

SOURCE:  data/evaluation_results/qwen_7B_NoQuant_FT.csv
SOURCE:  data/evaluation_results/llama_13B_8bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_14B_8bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama13B_NoQuant_FT_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_7B_8bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_14B_4bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_13B_NoQuantbit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_7B_NoQuant_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_7B_4bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/mistral_4bits_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_7B_NoQuantbit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/zefiro_NoQuant_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_r

,file,f1_score,precision,recall,model_type,instructed,model_configurations,model_size,quantization,fine_tuning,maxNewTokensFactor,nShotsInference,model,bnb_4bit_compute_dtype,r,lora_alpha,lora_dropout,gradient_accumulation_steps,learning_rate
0,data/qwen/7B_NoQuant_FT/maxNewTokensFactor2_nS...,0.382627,0.593101,0.282409,qwen,instructed,7B_NoQuant_FT,7,NoQuant,FT,2,2,qwen1.5-7b-chat_,torch.bfloat16,32.0,64.0,0.01,8.0,0.0002
1,data/qwen/7B_NoQuant_FT/maxNewTokensFactor2_nS...,0.380591,0.583910,0.282295,qwen,instructed,7B_NoQuant_FT,7,NoQuant,FT,2,2,qwen1.5-7b-chat_,torch.bfloat16,16.0,32.0,0.01,2.0,0.0002
2,data/qwen/7B_NoQuant_FT/maxNewTokensFactor2_nS...,0.365971,0.563712,0.270932,qwen,instructed,7B_NoQuant_FT,7,NoQuant,FT,2,4,qwen1.5-7b-chat_,torch.bfloat16,32.0,64.0,0.01,2.0,0.0002
3,data/qwen/7B_NoQuant_FT/maxNewTokensFactor4_nS...,0.489239,0.570002,0.428523,qwen,instructed,7B_NoQuant_FT,7,NoQuant,FT,4,2,qwen1.5-7b-chat_,torch.bfloat16,16.0,32.0,0.01,2.0,0.0002
4,data/qwen/7B_NoQuant_FT/maxNewTokensFactor2_nS...,0.402171,0.606623,0.300794,qwen,instructed,7B_NoQuant_FT,7,NoQuant,FT,2,0,qwen1.5-7b-chat_,torch.bfloat16,32.0,64.0,0.01,4.0,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,0.457107,0.555305,0.388421,llama,instructed,7B_8bit_base,7,8bit,base,8,3,BaseModel,NaN,NaN,NaN,NaN,NaN,NaN
11,data/llama/7B_8bit_base/maxNewTokensFactor2_nS...,0.366001,0.642696,0.255852,llama,instructed,7B_8bit_base,7,8bit,base,2,3,BaseModel,NaN,NaN,NaN,NaN,NaN,NaN
12,data/llama/7B_8bit_base/maxNewTokensFactor2_nS...,0.251434,0.664852,0.155032,llama,instructed,7B_8bit_base,7,8bit,base,2,2,BaseModel,NaN,NaN,NaN,NaN,NaN,NaN
13,data/llama/7B_8bit_base/maxNewTokensFactor2_nS...,0.276631,0.625473,0.177587,llama,instructed,7B_8bit_base,7,8bit,base,2,1,BaseModel,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
for col in res.columns:
    if col in ['file', 'f1_score', 'recall', 'precision', 'model_configurations', 'model']:
        continue
    print('col: ', col, 'vals: ', res[col].unique())

col:  model_type vals:  ['zefiro' 'mistral' 'qwen' 'llama']
col:  instructed vals:  ['instructed' 'notInstructed']
col:  model_size vals:  ['7' '14' '13']
col:  quantization vals:  ['4bit' '8bit' 'NoQuant']
col:  fine_tuning vals:  ['FT' 'unsure' 'base']
col:  maxNewTokensFactor vals:  ['8' '4' '2']
col:  nShotsInference vals:  ['4' '0' '2' '1' '3']
col:  bnb_4bit_compute_dtype vals:  ['torch.bfloat16' nan]
col:  r vals:  [32. 64. 16. nan]
col:  lora_alpha vals:  [64. 32. nan]
col:  lora_dropout vals:  [0.01  nan 0.05]
col:  gradient_accumulation_steps vals:  [ 4.  8.  2. nan 16.]
col:  learning_rate vals:  ['0.0002' nan '0.0008' '0.002']
